# Food.com Dataset Cleaning
In this notebook, we clean the raw Food.com dataset file and convert:
* ingredient parts
* ingredient quantities
* instructions 

from R-style vector strings into structured Python lists.

We then keep only the columns needed for the CookMate RAG pipeline.

In [1]:
import pandas as pd
import numpy as np 
import os

df = pd.read_csv("../data/raw/recipes.csv")
df.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


We define a helper function that:
* checks for missing values
* strips the surrounding `c(...)`
* extracts all strings inside quotes using a regex
* returns a clean Python list of strings.

In [2]:
import re

def parse_r_vector_string(s):
    """
    Convert R-style vector string like:
        c("a", "b", "c")
    into a Python list: ["a", "b", "c"]
    """
    if pd.isna(s):
        return []
    
    s = s.strip()

    # If the string doesn't start with c(, maybe it is already cleaned or has an odd format 
    if not s.startswith("c("):
        # We trye to split it by "," in quotes anyway
        # and we return it as a single-element list
        return [s]
    
    # We remove "c(" from the start and ")" from the end
    inner = s[2:-1]

    # Now inner looks like: "a", "b", "c"
    # We find all the substrings inside the quotes
    items = re.findall(r'"(.*?)"', inner)

    # At the end we strip the spaces
    return [item.strip() for item in items]


We now parse both the ingredient names and their corresponding quantities into:
* `ingredients_list` - list of ingredient names
* `quantities_list` - list of amounts

These lists will later be used to build structured ingredient entries in the LLM output or in the RAG context.

In [4]:
df["ingredients_list"] = df["RecipeIngredientParts"].apply(parse_r_vector_string)
df["quantities_list"] = df["RecipeIngredientQuantities"].apply(parse_r_vector_string)

In [6]:
df[["RecipeIngredientParts", "ingredients_list",
    "RecipeIngredientQuantities", "quantities_list"]].head(3)

,RecipeIngredientParts,ingredients_list,RecipeIngredientQuantities,quantities_list
0,"c(""blueberries"", ""granulated sugar"", ""vanilla ...","[blueberries, granulated sugar, vanilla yogurt...","c(""4"", ""1/4"", ""1"", ""1"")","[4, 1/4, 1, 1]"
1,"c(""saffron"", ""milk"", ""hot green chili peppers""...","[saffron, milk, hot green chili peppers, onion...","c(""1"", ""4"", ""2"", ""2"", ""8"", ""1/4"", ""8"", ""1/2"", ...","[1, 4, 2, 2, 8, 1/4, 8, 1/2, 1, 1, 1/4, 1/4, 1..."
2,"c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...","[sugar, lemons, rind of, lemon, zest of, fresh...","c(""1 1/2"", ""1"", NA, ""1 1/2"", NA, ""3/4"")","[1 1/2, 1, 1 1/2, 3/4]"


We now parse `RecipeInstructions` into a `steps_list`, which is a Python list where each element is a step.
We also compute:
* `n_steps_clean` = number of steps per recipe
* `n_ingredients_clean` = number of ingredients per recipe

These will help us filter out low-quality recipes (e.g., 0 steps, 0 ingredients).

In [7]:
df["steps_list"] = df['RecipeInstructions'].apply(parse_r_vector_string)

In [8]:
df[["RecipeInstructions", "steps_list"]].iloc[0]

RecipeInstructions    c("Toss 2 cups berries with sugar.", "Let stan...
steps_list            [Toss 2 cups berries with sugar., Let stand fo...
Name: 0, dtype: object

In [10]:
df["n_steps_clean"] = df["steps_list"].apply(len)
df["n_ingredients_clean"] = df["ingredients_list"].apply(len)

df[["n_steps_clean", "n_ingredients_clean"]].describe()

,n_steps_clean,n_ingredients_clean
count,522517.000000,522517.000000
mean,6.684380,7.907213
std,4.317089,3.938317
min,1.000000,1.000000
25%,4.000000,5.000000
50%,6.000000,7.000000
75%,9.000000,10.000000
max,107.000000,44.000000


We remove recipes that:
* have zero ingredients after parsing
* have zero steps after parsing
* have extremely long instructions, which may be outliers

This improves overall dataset quality for embeddings and retrieval.

In [11]:
mask_valid = (df["n_steps_clean"] > 0) & (df["n_ingredients_clean"] > 0)
df_clean = df[mask_valid].copy()

len_before = len(df)
len_after = len(df_clean)

len_before, len_after

(522517, 522517)

In [12]:
df_clean["steps_char_len"] = df_clean["RecipeInstructions"].apply(len)
df_clean["steps_char_len"].describe()

count    522517.000000
mean        594.262506
std         432.607030
min           2.000000
25%         314.000000
50%         495.000000
75%         754.000000
max       12709.000000
Name: steps_char_len, dtype: float64

In [13]:
df_clean = df_clean[df_clean["steps_char_len"] < 10000]
len(df_clean)

522513

We create a compact cleaned DataFrame that contains:
* `recipe_id` - unique recipe identifier
* `title` - recipe name
* `ingredients_list` - list of ingredient names
* `quantities_list` - list of corresponding quantities
* `steps_list` - list of instruction steps
* `Calories`, `FatContent`, `CarbohydrateContent`, `ProteinContent` - for nutrition estimation
* `RecipeCategory`, `Keywords` - for cuisine or tag filtering

This cleaned DataFrae will be used for:
* building embeddings
* RAG retrieval
* basic nutrition calculations.

In [15]:
cols_to_keep = [
    "RecipeId",
    "Name",
    "ingredients_list",
    "quantities_list",
    "steps_list",
    "Calories",
    "FatContent",
    "CarbohydrateContent",
    "ProteinContent",
    "RecipeCategory",
    "Keywords"
]

df_final = df_clean[cols_to_keep].copy()
df_final.rename(columns={
    "RecipeId": "recipe_id",
    "Name": "title"
}, inplace=True)

df_final.head()

,recipe_id,title,ingredients_list,quantities_list,steps_list,Calories,FatContent,CarbohydrateContent,ProteinContent,RecipeCategory,Keywords
0,38,Low-Fat Berry Blue Frozen Dessert,"[blueberries, granulated sugar, vanilla yogurt...","[4, 1/4, 1, 1]","[Toss 2 cups berries with sugar., Let stand fo...",170.9,2.5,37.1,3.2,Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",..."
1,39,Biryani,"[saffron, milk, hot green chili peppers, onion...","[1, 4, 2, 2, 8, 1/4, 8, 1/2, 1, 1, 1/4, 1/4, 1...",[Soak saffron in warm milk for 5 minutes and p...,1110.7,58.8,84.4,63.4,Chicken Breast,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",..."
2,40,Best Lemonade,"[sugar, lemons, rind of, lemon, zest of, fresh...","[1 1/2, 1, 1 1/2, 3/4]","[Into a 1 quart Jar with tight fitting lid, pu...",311.1,0.2,81.5,0.3,Beverages,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",..."
3,41,Carina's Tofu-Vegetable Kebabs,"[extra firm tofu, eggplant, zucchini, mushroom...","[12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1/2, 1/...","[Drain the tofu, carefully squeezing out exces...",536.1,24.0,64.2,29.3,Soy/Tofu,"c(""Beans"", ""Vegetable"", ""Low Cholesterol"", ""We..."
4,42,Cabbage Soup,"[plain tomato juice, cabbage, onion, carrots, ...","[46, 4, 1, 2, 1]","[Mix everything together and bring to a boil.,...",103.6,0.4,25.1,4.3,Vegetable,"c(""Low Protein"", ""Vegan"", ""Low Cholesterol"", ""..."


We save the cleaned dataset into:
* `data/cleaned/cleaned_recipes.json`
* `data/cleaned/cleaned_recipes.parquet`

The JSON file is convenient for portability, while Parquet is efficient for Python loading and vectorization.

In [16]:
os.makedirs("../data/cleaned", exist_ok=True)

In [17]:
df_final.to_json("../data/cleaned/cleaned_recipes.json", orient="records")

In [18]:
df_final.to_parquet("../data/cleaned/cleaned_recipes.parquet")